In [1]:
# Importing Qiskit
from qiskit import *
from qiskit.qasm2 import dumps
from qiskit_aer import Aer

import numpy as np
# import operation as op
from functools import reduce
import random
from random import randrange

# 1. Qubit Manipulation

In [2]:
def NoisyChannel(qc1, qc2, qc1_name, noise = 0.05):
    ''' This function takes the output of a circuit qc1 (made up only of x and 
        h gates), simulates a noisy quantum channel where Pauli errors (X - bit flip; Z - phase flip)
        will occur in qc2, and then initializes another circuit qc2 with the introduced noise.
    ''' 
    
    # Retrieve quantum state from qasm code of qc1
    qs = [dumps(qc1[i]).split('\n') for i in range(len(qc1))]
    
    # Process the code to get the instructions
    parsed_instructions = []
    for i, qasm_code in enumerate(qs):
        for line in qasm_code:
            line = line.strip()    # removing leading/trailing whitespace
            if line.startswith(('x', 'h', 'measure')):
                line = line.replace('0', str(i))
                parsed_instructions.append(line)
    
    # Apply parsed instructions to qc2
    for instruction in parsed_instructions:
        if instruction.startswith('x'):
            old_qr = int(instruction.split()[1][2:-2])
            qc2[old_qr].x(0)
            
        elif instruction.startswith('h'):
            old_qr = int(instruction.split()[1][2:-2])
            qc2[old_qr].h(0)
        
        elif instruction.startswith('measure'):
            continue    # exclude measuring
            
        else:
            print(f"Unable to parse instruction: {instruction}")
            raise Exception('Unable to parse instruction')
    
    # Introducing noise (taking input)
    for instruction in parsed_instructions:
        if random.random() < noise:
            old_qr = int(instruction.split()[1][2:-2])
            qc2[old_qr].x(0)     # Apply bit-flip error
            
        if random.random() < noise:
            old_qr = int(instruction.split()[1][2:-2])
            qc2[old_qr].z(0)     # Apply phase-flip error

In [3]:
def generate_random_bits(num):
    """This function generates a random bit-string of size = num"""
    
    bit_string = ""
    for _ in range(num):
        rand_bit = random.randint(0, 1)     # Flip Coin
        bit_string += str(rand_bit)
        
    return bit_string

In [4]:
def generate_random_bases(num_of_bases):
    """This function selects a random basis for each bit"""
    
    bases_string = ""
    for _ in range(num_of_bases):
        randBasis = random.randint(0, 1)     # Flip Coin

        if randBasis == 0:
            bases_string += "Z" 
        else:
            bases_string += "X"
            
    return bases_string

In [5]:
def encode(bits, bases):
    """This function encodes each bit into the given basis."""
    
    encoded_qubits = []
    
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1)     # Create a quantum circuit for each qubit
        
        # Possible Cases
        if bit == "1" :
            qc.x(0)

        if basis == 'X' :
            qc.h(0)
            
        encoded_qubits.append(qc)
            
    return (encoded_qubits)

In [6]:
def measure(qubits, bases):
    """This function measures each qubit in the corresponding basis chosen for it."""

    bits = ""    # The results of measurements

    for qubit, basis in zip(qubits, bases):

        if basis == "X" :
            qubit.h(0)
            
        qubit.measure(0, 0)
        
        # Execute on Simulator
        simulator = Aer.get_backend('qasm_simulator')
        transpiled_circuit = transpile(qubit, simulator)
        result = simulator.run(transpiled_circuit, shots=1).result()
        counts = result.get_counts()
        measured_bit = max(counts, key=counts.get)     # Max doesn't matter for simulator since there is only one shot.

        bits += str(measured_bit)
        
    return bits

# 2. Error Correction

In [7]:
help(reduce)

Help on built-in function reduce in module _functools:

reduce(...)
    reduce(function, sequence[, initial]) -> value
    
    Apply a function of two arguments cumulatively to the items of a sequence,
    from left to right, so as to reduce the sequence to a single value.
    For example, reduce(lambda x, y: x+y, [1, 2, 3, 4, 5]) calculates
    ((((1+2)+3)+4)+5).  If initial is present, it is placed before the items
    of the sequence in the calculation, and serves as a default when the
    sequence is empty.



In [8]:
def hamming(bob_bits, order):
    '''
    Takes a string of bits to be corrected (bob bits). Bit-wise sums the indices of elements which are '1'. The 0th bit stores the parity
    of the entire block. The location of the error is returned. 
    If the location is not '0', the current 0th parity is matched with that of the parity obtained after flipping the bit at the location obtained. 
    If the parity matches, then the error is found and corrected. If the parity doesn't match then there are more than 1 error.

    If the location is '0', then no error is present.
    
    '''
    
    loc = reduce(lambda x, y : x^y, [i for i, bit in enumerate(bob_bits) if bit == 1])    # x^y will apply xor to the binary rep of i -> index of 1s
    # loc = reduce(op.XOR, [i for i, bit in enumerate(bob_bits) if bit == 1])
    
    print(loc)
    bin_loc = bin(loc)[2:]
    binary_rep = f"{bin_rep(loc, order)}"

    par_0 = sum(bob_bits[i] for i in range(1, len(bob_bits)))%2

    if loc != 0 :
        if (par_0 + 1)%2 == bob_bits[0]:
            err_count = 1
            print(f"Error found at location : {loc}")

        else :
            err_count = 2
            print("More than 1 errors found")
            
    else : 
        err_count = 0
        print("No errors found")

    
    return err_count, loc, binary_rep

In [9]:
# Block is the representation of an array consisting the binary data string that has been reshaped as a square matrix
# It has dimensions : dim*dim, where dim = 2**(order/2)

def bin_rep(loc, order):
    '''
    Takes a number(int) and order/precision(int) as an input, and returns the binary form with the requested precision.
    '''
    bin_loc = bin(loc)[2:]
    bin_rep = f"{'0'*(order - len(bin_loc))}{bin_loc}"
    
    return bin_rep


def parity(order):
    '''
    Takes in order(int) as a parameter. Returns 2 arrays : 
       - parity_bits : an array containing '0' and the powers of 2 till 2^(order-1)
       - bin_parity : an array of the binary representation of elements of parity_bits   
    '''
    PARITY_DICT = {0:0, **{2**i : 0 for i in range(order)}}
    # parity_bits = np.array([0] + [2**i for i in range(order)]).astype(int)
    bin_parity = np.array([bin_rep(int(i), int(order)) for i in PARITY_DICT.keys()])

    return PARITY_DICT, bin_parity
    
    
def parity_locs(order):
    '''
    Takes in order(int) as a parameter. Returns an array :
        - parity_locs : A block(array reshaped as square matrix) with 1 at the locations of parity bits
    '''
    parity_locs = np.full(2**order, '-', dtype = object)
    PARITY_DICT = parity(order)[0]
    
    for loc in PARITY_DICT.keys() : parity_locs[loc] = '1'

    return parity_locs


"""
def print_extras(choice = 0):

    if choice == 0 : choice = [i in range(4)] 
    
    if '1' in choice :
        print(f"Parity Dictionary : {PARITY_DICT} \nBinary representation of parity bit indices : {bin_parity}")

    if '2' in choice :
        print(f"\nShape of the block : {dim}*{dim}")
        print("Uncorrected Alice block : \n", alice_bits.reshape(dim, dim))
        print(f'Parity bit locations : \n{parity_locs(order).reshape(dim, dim)}')

    if '3' in choice : 
        print(highlight.reshape(dim, dim))
"""

'\ndef print_extras(choice = 0):\n\n    if choice == 0 : choice = [i in range(4)] \n    \n    if \'1\' in choice :\n        print(f"Parity Dictionary : {PARITY_DICT} \nBinary representation of parity bit indices : {bin_parity}")\n\n    if \'2\' in choice :\n        print(f"\nShape of the block : {dim}*{dim}")\n        print("Uncorrected Alice block : \n", alice_bits.reshape(dim, dim))\n        print(f\'Parity bit locations : \n{parity_locs(order).reshape(dim, dim)}\')\n\n    if \'3\' in choice : \n        print(highlight.reshape(dim, dim))\n'

### Creating some dummy data for Alice

In [10]:
order = 6
dim = int(2**(order/2))
alice_bits = np.array([random.randint(0, 1) for _ in range(2**order)])    # Randomly fills the array with 0/1
print(f" Alice Block : \n{alice_bits.reshape(dim, dim)}")

 Alice Block : 
[[0 0 1 1 1 0 0 1]
 [1 0 1 1 0 1 0 0]
 [1 0 1 1 1 0 0 0]
 [1 0 0 0 0 1 1 0]
 [1 0 1 0 0 0 0 1]
 [1 0 1 0 0 1 0 0]
 [1 0 0 1 0 0 0 0]
 [0 1 0 0 0 1 1 0]]


## Encoding parity bits in Alice's key

In [11]:
def encode_parity(alice_bits, order):
    # order = np.ceil(np.log2(len(alice_bits)))
    PARITY_DICT, bin_parity = parity(order)
    # print_extras(choice = ['2', '1'])
    
    sub_block = int(2**(order - 1))
    parity_of = np.zeros((len(PARITY_DICT), sub_block)).astype(int)   # An array to store the locations affecting the parity p
    
    for p in range(1, order+1) :    # checking for 1 at position p. eg : bin(45) = 101101
    
        bit_index = 2**(p-1)
        highlight = np.zeros(2**order).astype(int)                        # Highlights the locations affected by the current parity bit
        # print(f"bin rep of {bit_index = } : {bin_parity[p]}")
        
        for i in range(sub_block):                                         #  Order-1 = 5. range(5) = 0, 1, 2, 3, 4 => order-2
            bin_index = bin_rep(i, order-1)                                # Index(in binary formin binary form) for the data bits : 5 digits : 00010
            bin_index = bin_index[: order-p] + '1' + bin_index[order-p :]
            index = int(bin_index, base = 2)                                # Gives the index(int) of the elements to be considered for the current parity element
            
            parity_of[p, i] = index
            highlight[index] = 1
    
        PARITY_DICT[bit_index] = np.mod( sum( alice_bits[parity_of[p, i]] for i in range(sub_block) if bit_index != parity_of[p, i] ), 2 )
    
        if PARITY_DICT[bit_index] != alice_bits[bit_index] : 
            alice_bits[bit_index] = np.mod(alice_bits[bit_index] + 1, 2)
            
        # print_extras('3')    # print(highlight.reshape(dim, dim))
    
    PARITY_DICT[0] =  sum( i for i in range(1, 2**order) )%2
    # print(f"Parity locations : \n{parity_of[1:]}")
    
    print(hamming(alice_bits, order))
    print(alice_bits.reshape(dim, dim))

    return alice_bits, PARITY_DICT

In [12]:
encode_parity(alice_bits, order)

0
No errors found
(0, 0, '000000')
[[0 1 0 1 1 0 0 1]
 [0 0 1 1 0 1 0 0]
 [1 0 1 1 1 0 0 0]
 [1 0 0 0 0 1 1 0]
 [0 0 1 0 0 0 0 1]
 [1 0 1 0 0 1 0 0]
 [1 0 0 1 0 0 0 0]
 [0 1 0 0 0 1 1 0]]


(array([0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0,
        0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0]),
 {0: 0, 1: 1, 2: 0, 4: 1, 8: 0, 16: 1, 32: 0})

# Bob

In [13]:
bob_bits = np.copy(alice_bits)
num_errors = 1    # To decide for simulation : could be 0, 1, 2, or more
# pos = [random.randrange(0, 16) for _ in range(num_errors)]
pos = [5]    # For simulation purposes, keep it fixed. Else use the above commented statement
print(f"{num_errors=}, {pos=}")
# pos = [5, 7]
# bob_bits = np.array([not bob_bits[i] for i in pos])
bob_bits[pos] = np.mod(bob_bits[pos]+1, 2)
print(f"\nAlice :\n {alice_bits.reshape(dim, dim)} \n\nBob :\n {bob_bits.reshape(dim, dim)}")

num_errors=1, pos=[5]

Alice :
 [[0 1 0 1 1 0 0 1]
 [0 0 1 1 0 1 0 0]
 [1 0 1 1 1 0 0 0]
 [1 0 0 0 0 1 1 0]
 [0 0 1 0 0 0 0 1]
 [1 0 1 0 0 1 0 0]
 [1 0 0 1 0 0 0 0]
 [0 1 0 0 0 1 1 0]] 

Bob :
 [[0 1 0 1 1 1 0 1]
 [0 0 1 1 0 1 0 0]
 [1 0 1 1 1 0 0 0]
 [1 0 0 0 0 1 1 0]
 [0 0 1 0 0 0 0 1]
 [1 0 1 0 0 1 0 0]
 [1 0 0 1 0 0 0 0]
 [0 1 0 0 0 1 1 0]]


In [14]:
out = hamming(bob_bits, order)
print("Error counts : ", out[0], ", loc : ", out[1:])
# type(out[0]), print(out[0])
print(f"\nAlice :\n {alice_bits.reshape(dim, dim)} \n\nBob :\n {bob_bits.reshape(dim, dim)}")
# print(f'{out[0]:04b}')

5
Error found at location : 5
Error counts :  1 , loc :  (5, '000101')

Alice :
 [[0 1 0 1 1 0 0 1]
 [0 0 1 1 0 1 0 0]
 [1 0 1 1 1 0 0 0]
 [1 0 0 0 0 1 1 0]
 [0 0 1 0 0 0 0 1]
 [1 0 1 0 0 1 0 0]
 [1 0 0 1 0 0 0 0]
 [0 1 0 0 0 1 1 0]] 

Bob :
 [[0 1 0 1 1 1 0 1]
 [0 0 1 1 0 1 0 0]
 [1 0 1 1 1 0 0 0]
 [1 0 0 0 0 1 1 0]
 [0 0 1 0 0 0 0 1]
 [1 0 1 0 0 1 0 0]
 [1 0 0 1 0 0 0 0]
 [0 1 0 0 0 1 1 0]]


# Start

In [ ]:
# eve_presence = True
# Data_length = [1024, 512, 256, 128]
# parity_bits = [10 + 1, 9 + 1, 8 + 1, 7 + 1]
# KEY_LENGTH =  Data_length - parity_bits(Data_length)
# ch_noise = 2e-4
DATA_LENGTH = 512
PARITY_BITS = np.log2(DATA_LENGTH)
KEY_LENGTH = DATA_LENGTH - PARITY_BITS
# Preparation for encoding
random.seed(0)    # Seed the random number generator. This will be used as our "coin flipper"

print(KEY_LENGTH, DATA_LENGTH, PARITY_BITS)

## Alice

In [ ]:
# Generating a random string of bits
# If KEY_RESERVOIR exists, then alice_bits = KEY_RESERVOIR[:KEY_LENGTH]
alice_bits = generate_random_bits(KEY_LENGTH)
alice_bases = generate_random_bases(KEY_LENGTH) # Alice randomly chooses a basis for each bit.

### Encoding

In [ ]:
# Encode Alice's bits
encoded_qubits = encode(alice_bits, alice_bases)

### Parity

In [ ]:
order = np.ceil(np.log2(len(alice_bits)))
dim = int(2**(order/2))

print(f" Alice Block : \n{alice_bits.reshape(dim, dim)}")

In [ ]:
print('Alice Block after correction : ')
encode_parity(alice_bits, order)

# Quantum Signal Channel

## Eve
0 : Not present,    1 : present

In [ ]:
if eve_presence == 'Random': eve = random.randint(0, 1)
else: eve = int(eve_presence)
    
label = 'Eve' if eve else 'Alice'

In [ ]:
qubits_received = [QuantumCircuit(1, 1) for _ in range(len(encoded_qubits))]

if eve : 
    #print("Eve Present!")
    qubits_intercepted = [QuantumCircuit(1, 1) for _ in range(len(encoded_qubits))]
    
    NoisyChannel(encoded_qubits, qubits_intercepted, 'alice', noise = ch_noise) ##Eve intercepts noisy states     

    eve_bases = generate_random_bases(KEY_LENGTH) # Generate a random set of bases
    eve_bits = measure(qubits_intercepted, eve_bases) # Measure the qubits
    
    # Eve encodes her decoy qubits and sends them along the quantum channel    
    encoded_intercepted_qubits = encode(eve_bits, eve_bases)    
    NoisyChannel(encoded_intercepted_qubits, qubits_received, 'Eve', noise = ch_noise) ## Eve sends noisy states to Bob

else : 
    NoisyChannel(encoded_qubits, qubits_received, 'Alice', noise = ch_noise) ## Alice sends noisy states to Bob


## Bob

In [ ]:
bob_bases = generate_random_bases(KEY_LENGTH) # Bob randomly chooses a basis for each bit.

# Measurement
bob_bits = measure(qubits_received, bob_bases)

# Public Interaction Channel

## Sifting

In [ ]:
BROADCAST = alice_bases    # Alice tells Bob which bases she used. BROADCAST uses classical channel

# Store the indices of the bases they share in common
common_bases = [i for i in range(KEY_LENGTH) if BROADCAST[i] == bob_bases[i]]

In [ ]:
bob_bits = [bob_bits[index] for index in common_bases]
BROADCAST = common_bases    # Bob tells Alice which bases they shared in common
alice_bits = [alice_bits[index] for index in BROADCAST]    # Alice keeps only the bits they shared in common

## Reconciliation 
(Comparision -- Spotting)

In [ ]:
hamming(bob_bits, aice_bits)    # Assuming single bit error

sample = len(alice_bits)//3    # len(alice_bits) >= 3
errors = 0

for _ in range(sample):
    bit_index = random.randrange(len(alice_bits)) 
    
    if alice_bits[bit_index] != bob_bits[bit_index]:  errors += 1    #calculating errors
        
    del alice_bits[bit_index] #removing tested bits from key strings
    del bob_bits[bit_index]

# QBER

In [ ]:
QBER = round(errors/sample, 4) # calculating QBER and saving the answer to two decimal places
QBER

## Final Key : 

In [ ]:
if QBER > error_threshold:
    if eve : print(" Eve detected : ", end = " ")
    else : print(" Eve FALSELY not detected : ", end = " ")
    print("Key not secure")

else :
    if eve : print('Eve went unnoticed : ', end = " ")
    else : print('Eve not present : ', end = " ")
    print("Key is secure")


In [ ]:
key = "" 
for bit in alice_bits:    # Or bob_bits, since both should be the same
    key += bit

In [ ]:
errors, sample